**Importing Libraries**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
 # importing the necessary libs
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import io
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
#from tensorflow.keras.utils import to_categorical
#from tensorflow.keras.applications import EfficientNetB4
#from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from keras.utils import plot_model
from keras.utils import to_categorical
from keras.applications import EfficientNetB4
import tensorflow as tf
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
from keras.utils import plot_model
import ipywidgets as widgets
from PIL import Image
from IPython.display import display, clear_output

In [ ]:
class_names = ['Final AD JPEG', 'Final CN JPEG', 'Final MCI JPEG']

In [ ]:
image_size = 150

**Preparing imgaes from dataset**

In [ ]:
# Prepare the training images and labels
train_images = []
train_labels = []
for class_name in class_names:

    base_path='/content/drive/MyDrive/adni_dataset/project_dataset/adni/Alzheimers-ADNI'
    sub_path = 'train'
    folder_path = os.path.join(base_path,sub_path,class_name)
    for image_file in tqdm(os.listdir(folder_path)):
        image = cv2.imread(os.path.join(folder_path, image_file))
        image = cv2.resize(image, (image_size, image_size))
        train_images.append(image)
        train_labels.append(class_name)

100%|██████████| 463/463 [00:12<00:00, 38.21it/s]


In [ ]:
test_images = []
test_labels = []

for class_name in class_names:
    base_path='/content/drive/MyDrive/adni_dataset/project_dataset/adni/Alzheimers-ADNI'
    sub_path = 'test'
    folder_path = os.path.join(base_path,sub_path,class_name)
    for image_file in tqdm(os.listdir(folder_path)):
        image = cv2.imread(os.path.join(folder_path, image_file))
        image = cv2.resize(image, (image_size, image_size))
        test_images.append(image)
        test_labels.append(class_name)

100%|██████████| 82/82 [00:21<00:00,  3.86it/s]


In [ ]:
# Convert the training images and labels to numpy arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [ ]:
# Randomly shuffle the dataset
train_images, train_labels = shuffle(train_images, train_labels, random_state=42)

# Split the dataset into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(
    train_images, train_labels, test_size=0.1, random_state=42
)

**Encoding the labels**

In [ ]:
# Encode the training and test labels
train_labels_encoded = []
test_labels_encoded = []

for label in train_labels:
    train_labels_encoded.append(class_names.index(label))

train_labels = to_categorical(train_labels_encoded)

for label in test_labels:
    test_labels_encoded.append(class_names.index(label))

test_labels = to_categorical(test_labels_encoded)

**Model Creation**

In [ ]:
efficientnetB4 = EfficientNetB4(include_top=False, weights='imagenet', input_shape=(image_size, image_size, 3))

71686520/71686520 [==============================] - 1s 0us/step


In [ ]:
model = efficientnetB4.output
model = tf.keras.layers.GlobalAveragePooling2D()(model)
model = tf.keras.layers.Dropout(0.5)(model)
model = tf.keras.layers.Dense(len(class_names), activation='softmax')(model)
model = tf.keras.models.Model(inputs=efficientnetB4.input, outputs=model)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 150, 150, 3)          0         ['input_1[0][0]']             
                                                                                                  
 normalization (Normalizati  (None, 150, 150, 3)          7         ['rescaling[0][0]']           
 on)                                                                                              
                                                                                                  
 rescaling_1 (Rescaling)     (None, 150, 150, 3)          0         ['normalization[0][0]']   

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

**Creating checkpoint**

In [ ]:
tensorBoard = TensorBoard(log_dir="logs")
model_save_path = 'efficientNetB4.h5'
checkPoint = ModelCheckpoint(model_save_path,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True,
                             mode='auto')

reduce_lr = ReduceLROnPlateau(monitor='val_acc',
                             factor=0.3,
                             patience=2,
                             verbose=1,
                             mode='auto',
                             min_delta=0.001)

**Fitting train images and running the model for 50 iterations**

In [ ]:
history = model.fit(train_images,
                    train_labels,
                    batch_size=32,
                    validation_split=0.1,
                    epochs=50,
                    verbose=1,
                    callbacks=[tensorBoard, checkPoint, reduce_lr])

Epoch 1/50
29/29 [==============================] - ETA: 0s - loss: 1.0256 - accuracy: 0.5039 

29/29 [==============================] - 367s 13s/step - loss: 1.0256 - accuracy: 0.5039 - val_loss: 0.9867 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 2/50
29/29 [==============================] - ETA: 0s - loss: 0.8918 - accuracy: 0.5862 

29/29 [==============================] - 360s 12s/step - loss: 0.8918 - accuracy: 0.5862 - val_loss: 0.9675 - val_accuracy: 0.5400 - lr: 0.0010
Epoch 3/50
29/29 [==============================] - ETA: 0s - loss: 0.7286 - accuracy: 0.6696 

29/29 [==============================] - 367s 13s/step - loss: 0.7286 - accuracy: 0.6696 - val_loss: 2.1159 - val_accuracy: 0.4700 - lr: 0.0010
Epoch 4/50
29/29 [==============================] - ETA: 0s - loss: 0.5204 - accuracy: 0.8076 

29/29 [==============================] - 373s 13s/step - loss: 0.5204 - accuracy: 0.8076 - val_loss: 1.5414 - val_accuracy: 0.5500 - lr: 0.0010
Epoch 5/50
29/29 [==============================] - ETA: 0s - loss: 0.5300 - accuracy: 0.7853 

...

29/29 [==============================] - 356s 12s/step - loss: 0.1626 - accuracy: 0.9455 - val_loss: 2.3367 - val_accuracy: 0.4200 - lr: 0.0010
Epoch 47/50
29/29 [==============================] - ETA: 0s - loss: 0.1576 - accuracy: 0.9388 

29/29 [==============================] - 317s 11s/step - loss: 0.1576 - accuracy: 0.9388 - val_loss: 2.1998 - val_accuracy: 0.5300 - lr: 0.0010
Epoch 48/50
29/29 [==============================] - ETA: 0s - loss: 0.0590 - accuracy: 0.9800 

29/29 [==============================] - 327s 11s/step - loss: 0.0590 - accuracy: 0.9800 - val_loss: 2.3537 - val_accuracy: 0.5500 - lr: 0.0010
Epoch 49/50
29/29 [==============================] - ETA: 0s - loss: 0.0313 - accuracy: 0.9889 

29/29 [==============================] - 318s 11s/step - loss: 0.0313 - accuracy: 0.9889 - val_loss: 3.1083 - val_accuracy: 0.5200 - lr: 0.0010
Epoch 50/50
29/29 [==============================] - ETA: 0s - loss: 0.0232 - accuracy: 0.9933 

29/29 [==============================] - 312s 11s/step - loss: 0.0232 - accuracy: 0.9933 - val_loss: 2.9131 - val_accuracy: 0.5300 - lr: 0.0010


In [ ]:
!pip install keras-tuner


**code for model to apply hyper parameter tuning**

In [ ]:
import keras_tuner as kt
import tensorflow as tf

def build_model(hp):
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Conv2D(
        filters=hp.Int('filters', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('kernel_size', values=[3, 5]),
        activation='relu',
        input_shape=(150, 150, 3)
    ))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(
        units=hp.Int('units', min_value=32, max_value=128, step=16),
        activation='relu'
    ))
    model.add(tf.keras.layers.Dense(3, activation='softmax'))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model


In [ ]:
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='/content/drive/My Drive/intro_to_kt',  # Save the tuner results in Google Drive
    project_name='hyperband_project'
)


Reloading Tuner from /content/drive/My Drive/intro_to_kt/hyperband_project/tuner0.json


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/adni_dataset/project_dataset/adni/Alzheimers-ADNI/train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)


Found 1111 images belonging to 3 classes.


In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255)

val_generator = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/adni_dataset/project_dataset/adni/Alzheimers-ADNI/test',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)


Found 195 images belonging to 3 classes.


In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="/content/drive/My Drive/logs")


tuner.search(
    train_generator,
    validation_data=val_generator,
    epochs=5,
    callbacks=[tensorboard_callback]
)


best_model = tuner.get_best_models(num_models=1)[0]


In [ ]:
tuner.search(train_images, train_labels, epochs=10)




In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split


train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.2, random_state=42)

datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = datagen.flow(train_images, train_labels, batch_size=32)


val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow(val_images, val_labels, batch_size=32)


In [ ]:
import keras_tuner as kt
from tensorflow.keras.callbacks import TensorBoard

# Define your model as a HyperModel
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(
        filters=hp.Int('filters', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('kernel_size', values=[3, 5]),
        activation='relu',
        input_shape=(28, 28, 1)
    ))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(
        units=hp.Int('units', min_value=32, max_value=128, step=16),
        activation='relu'
    ))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


**Prediction of images **

In [ ]:
def predict_image(Imagename):
    print("started")

    #for name, fileinfo in upload.value.items():
    image = Image.open(Imagename)


    image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    image = cv2.resize(image, (image_size, image_size))
    image = image.reshape(1, image_size, image_size, 3)
    prediction = model.predict(image)
    prediction = np.argmax(prediction, axis=1)[0]

    if prediction == 0:
        prediction = "Final AD JPEG"
    elif prediction == 1:
        prediction = "Final CN JPEG"
    elif prediction == 2 :
        prediction = "Final MCI JPEG"
    print(f"Model predicts that it is a {prediction}")

In [ ]:
predictions = model.predict(test_images)
predictions = np.argmax(predictions, axis=1)
actual_labels = np.argmax(test_labels, axis=1)

3/3 [==============================] - 12s 2s/step


In [ ]:
predict_image('/content/drive/MyDrive/adni_dataset/project_dataset/adni/Alzheimers-ADNI/test/Final AD JPEG/ADNI_002_S_4521_MR_Axial_T2-Star__br_raw_20160324122521837_22_S402609_I662501.jpg')
predict_image('/content/drive/MyDrive/adni_dataset/project_dataset/adni/Alzheimers-ADNI/test/Final CN JPEG/ADNI_002_S_6007_MR_Axial_T2_STAR__br_raw_20180423110908167_22_S679010_I988541.jpg')
predict_image('/content/drive/MyDrive/adni_dataset/project_dataset/adni/Alzheimers-ADNI/train/Final MCI JPEG/ADNI_002_S_1155_MR_Axial_T2-Star__br_raw_20150106121009713_22_S244439_I467647.jpg')
predict_image('/content/drive/MyDrive/adni_dataset/project_dataset/adni/Alzheimers-ADNI/test/Final AD JPEG/ADNI_002_S_4521_MR_Axial_T2-Star__br_raw_20160324122521837_22_S402609_I662501.jpg')
predict_image('/content/drive/MyDrive/adni_dataset/project_dataset/adni/Alzheimers-ADNI/test/Final CN JPEG/ADNI_002_S_6007_MR_Axial_T2_STAR__br_raw_20180423110908167_22_S679010_I988541.jpg')
predict_image('/content/drive/MyDrive/adni_dataset/project_dataset/adni/Alzheimers-ADNI/test/Final CN JPEG/ADNI_002_S_6007_MR_Axial_T2_STAR__br_raw_20180423110908167_22_S679010_I988541.jpg')

started
1/1 [==============================] - 0s 147ms/step
Model predicts that it is a Final AD JPEG
started
1/1 [==============================] - 0s 135ms/step
Model predicts that it is a Final CN JPEG
started
1/1 [==============================] - 0s 133ms/step
Model predicts that it is a Final MCI JPEG
started
1/1 [==============================] - 0s 154ms/step
Model predicts that it is a Final AD JPEG
started
1/1 [==============================] - 0s 163ms/step
Model predicts that it is a Final CN JPEG
started
1/1 [==============================] - 0s 141ms/step
Model predicts that it is a Final MCI JPEG
started
1/1 [==============================] - 0s 135ms/step
Model predicts that it is a Final CN JPEG
